In [1]:
%%writefile kmeans.py
import numpy as np
from sklearn.cluster import KMeans

class KMeansClusterer:
    def __init__(self, data, k=10):
        self.data = data
        self.k = k
        self.kmeans = KMeans(n_clusters=k)
        self.used_indices = []

    def fit(self):
        self.kmeans.fit(self.data)

    def get_top_indices(self, num_top_indices=100):
        clustered_indices = []
        resulting_indices = []
        for cluster_id in range(self.k):
            top_indices = []
            if cluster_id not in clustered_indices:
                cluster_distances = self.kmeans.transform(self.data)[:, cluster_id]
                cluster_indices = np.argsort(cluster_distances)

                for idx in cluster_indices:
                    if idx not in self.used_indices:
                        top_indices.append(idx)
                        self.used_indices.append(idx)

                    if len(top_indices) >= num_top_indices:
                      resulting_indices.extend(top_indices)
                      break

                clustered_indices.append(cluster_id)

        return resulting_indices

Writing kmeans.py


In [2]:
%%writefile alexnet.py

# Authors: rafik gouiaa <rafikgouiaaphd@gmail.com>, ...

from typing import Optional, Callable

from torchvision.models import alexnet
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
import torch.optim as Optimizer
from tqdm import tqdm

class AlexNet(object):
    """
    Encapsulate the pretrained alexnet model
    Parameters
    ----------
    n_classes : int, default(10)
        the new number of classes
    device: Optional[str] 'cuda' or 'cpu', default(None)
            if None: cuda will be used if it is available
    """

    def __init__(self, n_classes: int = 10, device: Optional[str] = None):

        self.n_classes = n_classes
        self.model = alexnet(pretrained=False, progress=True)
        self.model.eval()

        # ## change last layer to accept n_classes instead of 1000 classes
        self.model.classifier[6] = nn.Linear(4096, self.n_classes)

        # ## Add softmax layer to alexnet model
        self.model = nn.Sequential(self.model, nn.LogSoftmax(dim=1))

        if device is None:
            self.device = torch.device(
                "cuda:0" if torch.cuda.is_available() else "cpu")
        print('The code is running on {} '.format(self.device))


    def __train_one_epoch(self, train_loader: DataLoader,
                          optimizer: Optimizer,
                          criterion: Callable,
                          valid_loader: DataLoader = None,
                          epoch: int = 0,
                          each_batch_idx: int = 300) -> None:
        """
        Train alexnet for one epoch
        Parameters
        ----------
        train_loader : DataLoader
        criterion :  Callable
        optimizer : Optimizer (torch.optim)
        epoch : int
        each_batch_idx : int
            print training stats after each_batch_idx

        Returns
        -------
        None
        """
        train_loss = 0
        data_size = 0
        print("Training ...")
        for batch_idx, sample_batched in tqdm(enumerate(train_loader)):
            # load data and label
            data, label = sample_batched

            # convert data and label to be compatible with the device
            data = data.to(self.device)
            data = data.float()
            label = label.to(self.device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # run forward
            pred_prob = self.model(data)

            # calculate loss
            loss = criterion(pred_prob, label)

            # calculate gradient (backprop)
            loss.backward()

            # total train loss
            train_loss += loss.item()
            data_size += label.size(0)

            # update weights
            optimizer.step()

            if batch_idx % each_batch_idx == 0:
                print('Train Epoch: {}, Train Size : {}\t Loss: {:.6f}'.format(
                    epoch,
                    len(train_loader.sampler.indices),
                    loss.item()))
        if valid_loader:
            acc = self.evaluate(test_loader=valid_loader)
            print('Accuracy on the valid dataset {}'.format(acc))

        print('====> Epoch: {} Average loss: {:.4f}'.
              format(epoch,
                     train_loss / data_size))

    def train(self, epochs: int, train_loader: DataLoader,
              valid_loader: DataLoader = None) -> None:
        """
        Train alexnet for several epochs
        Parameters
        ----------
        epochs : int
            number of epochs
        train_loader:  DataLoader
            training set
        valid_loader : DataLoader, Optional

        Returns
        -------
        None
        """
        self.model.to(self.device)
        self.model.train()
        # optimizer = optim.SGD(
        #     filter(lambda p: p.requires_grad, self.model.parameters()),
        #     lr=0.01, momentum=0.9)
        # optimizer = optim.Adam(
        #     filter(lambda p: p.requires_grad, self.model.parameters()),
        #     lr=0.01)
        optimizer = optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)

        criterion = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            self.__train_one_epoch(train_loader=train_loader,
                                   optimizer=optimizer,
                                   criterion=criterion,
                                   valid_loader=valid_loader,
                                   epoch=epoch
                                   )

    def evaluate(self, test_loader: DataLoader) -> float:
        """
        Calaculate alexnet accuracy on test data
        Parameters
        ----------
        test_loader: DataLoader

        Returns
        -------
        accuracy: float
        """
        correct = 0
        total = 0
        print("Evaluation ...")
        with torch.no_grad():
            for batch_idx, sample_batched in tqdm(enumerate(test_loader)):
                data, labels = sample_batched

                data = data.to(self.device)
                data = data.float()
                labels = labels.to(self.device)
                outputs = self.model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total

    def predict(self, test_loader):
        """
        Run the inference pipeline on the test_loader data
        Parameters
        ----------
        test_loader: DataLoader
            test data

        Returns
        -------

        """
        self.model.eval()
        self.model.to(self.device)
        predict_results = np.empty(shape=(0, 10))
        print("Prediction on Unlabelled Data ...")
        with torch.no_grad():
            for batch_idx, sample_batched in tqdm(enumerate(test_loader)):
                data, _ = sample_batched

                data = data.to(self.device)
                data = data.float()
                outputs = self.model(data)
                outputs = softmax(outputs)
                predict_results = np.concatenate(
                    (predict_results, outputs.cpu().numpy()))
        return predict_results

Writing alexnet.py


In [3]:
%%writefile utils.py
import torch
import random
import numpy as np

def random_sampler(selected_indices,n = 50000,sample_size = 2000) :
  random.seed(42)
  torch.manual_seed(42)
  np.random.seed(42)
  all_indices = list(range(n))

  all_indices = list(set(all_indices) -  set(selected_indices))

  sample_indices = random.sample(all_indices, sample_size)

  return sample_indices

Writing utils.py


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%writefile main.py

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.cluster import KMeans
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from torch.utils.data.sampler import SubsetRandomSampler
import random
import pickle

from kmeans import KMeansClusterer
from alexnet import AlexNet
from utils import random_sampler

import argparse

parser = argparse.ArgumentParser(description = """
                      max_run : maximum number of model runs
                      top_k : test Dataset size
                      dev_size : dev Dataset Size
                      model_type : name of model
                      """)
parser.add_argument('-runs',
            '--max_run',
            type = int,
            default = 2,
            help = 'maximum number of model runs')

parser.add_argument('-top_k',
            '--sample_size_per_cluster',
            type = int,
            default = 50,
            help = 'number of samples to be selected from each cluster')

parser.add_argument('-k',
            '--clusters',
            type = int,
            default = 10,
            help = 'Number of clusters')

parser.add_argument('-b',
            '--batch_size',
            type = int,
            default = 64,
            help = 'Batch size for training models')

parser.add_argument('-e',
            '--epochs',
            type = int,
            default = 40,
            help = 'Number of epochs for model training')

parser.add_argument('-method',
            '--sample_strategy',
            type = str,
            default = 'kmeans',
            help = 'Stratefy to select samples "random" or "kmeans"')

parser.add_argument('-d',
            '--dataset',
            type = str,
            default = 'cifar100',
            help = 'Dataset Name')

parser.add_argument('-n',
            '--n_classes',
            type = int,
            default = 100,
            help ='Number of classes')

parser.add_argument('-cf',
            '--cluster_flag',
            type = int,
            default = 1,
            help ='To do Clustering or not')

args = parser.parse_args()


if __name__ == "__main__" :
  random.seed(42)
  torch.manual_seed(42)
  np.random.seed(42)


  ## Parameters
  max_runs = args.max_run
  top_k = args.sample_size_per_cluster
  clusters = args.clusters
  batch_size = args.batch_size
  epochs = args.epochs
  method = args.sample_strategy
  dataset = args.dataset
  n_classes = args.n_classes
  cluster_flag = args.cluster_flag

  selected_indices = []
  acc_list = []


  if method == "random" :
    print("Starting Random Sampling Based Active Learning : ")
  elif method == "kmeans" :
    print("Starting Clustering Based Active Learning : ")

  # Step 1: Load the CIFAR-100 dataset
  transform = transforms.Compose([transforms.Resize(224),  # ViT model expects 224x224 images
                                  transforms.ToTensor()])

  if dataset == "cifar100" :
    cifar_train = datasets.CIFAR100(root="./data", train=True, transform=transform, download=True)
    cifar_val = datasets.CIFAR100(root="./data", train=False, transform=transform, download=True)

    cifar_train1 = datasets.CIFAR100(root="./data", train=True, transform=None, download=True)
    train_data_features = cifar_train1.data.reshape(cifar_train.data.shape[0], -1)
    train_data_labels = cifar_train1.targets

  elif dataset == "cifar10" :
    ## Initialization
    cifar_train = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
    cifar_val = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

    cifar_train1 = datasets.CIFAR100(root="./data", train=True, transform=None, download=True)
    train_data_features = cifar_train1.data.reshape(cifar_train.data.shape[0], -1)
    train_data_labels = cifar_train1.targets

  if (method == "kmeans") and (cluster_flag == 1) :
    print('Dimensionality Reduction started ...')
    pca = PCA(n_components=0.99)
    train_data_embed = pca.fit_transform(train_data_features)
    labels = train_data_labels

    ## Clustering Step
    print('Clustering Started ...')
    clusterer = KMeansClusterer(train_data_embed, k=clusters)
    clusterer.fit()

    ## Saving the Cluster object
    with open("/content/drive/MyDrive/Project Proposal/kmeans_pca_k20_c100.pkl", "wb") as f:
      pickle.dump(clusterer, f)
  elif (method == "kmeans") and (cluster_flag == 0) :
    with open("/content/drive/MyDrive/Project Proposal/kmeans_pca_k20_c100.pkl", "rb") as f:
      clusterer = pickle.load(f)


  transform = transforms.Compose([
      transforms.Resize(224),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  if method == "kmeans" :
    train_indices = clusterer.get_top_indices(top_k)
  elif method == "random" :
    train_indices = random_sampler(selected_indices,n = 50000,sample_size = top_k*clusters)
    selected_indices.extend(train_indices)

  val_indices = list(range(len(cifar_val)))


  train_sampler = SubsetRandomSampler(train_indices)
  valid_sampler = SubsetRandomSampler(val_indices)

  train_dataloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size,
                                    sampler=train_sampler, num_workers=4)
  val_dataloader = torch.utils.data.DataLoader(cifar_val, batch_size=batch_size,
                                      sampler=valid_sampler,num_workers=4)


  # Create the model
  model = AlexNet(n_classes=n_classes, device=None)

  for run in range(max_runs) :

    # Initialize the model
    print('Intialize training the model')

    model.train(epochs=epochs, train_loader=train_dataloader, valid_loader=None)
    PATH = '/content/drive/MyDrive/Project Proposal/model_dataset_{dataset}_train_size_{len(train_dataloader.dataset)}_method_{method}.pkl'
    torch.save(model, PATH)

    # Evaluate model on dtest
    acc = model.evaluate(test_loader=val_dataloader)
    acc_list.append(acc)
    print('====> Accuracy: {} '.format(acc))

    print("Updating the Training Data : ")

    if method == "kmeans" :
      new_train_indices = clusterer.get_top_indices(top_k)
    elif method == "random" :
      new_train_indices = random_sampler(selected_indices,n = 50000,sample_size = top_k*clusters)
      selected_indices.extend(new_train_indices)

    train_dataloader.sampler.indices.extend(new_train_indices)

    print("All accuracies : ")
    print(acc_list)

Writing main.py


In [28]:
%run main.py -runs 3 -d 'cifar10' -n 10 -top_k 2 -k 250 -cf 1

Starting Clustering Based Active Learning : 
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Dimensionality Reduction started ...
Clustering Started ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


The code is running on cuda:0 
Intialize training the model
Training ...


5it [00:00, 17.72it/s]

Train Epoch: 0, Train Size : 500	 Loss: 2.303176


...

Train Epoch: 39, Train Size : 1500	 Loss: 0.053042


24it [00:01, 22.15it/s]


====> Epoch: 39 Average loss: 0.0011
Evaluation ...


157it [00:04, 31.66it/s]


====> Accuracy: 43.77 
Updating the Training Data : 
All accuracies : 
[17.51, 39.78, 43.77]


In [29]:
%run main.py -runs 3 -method 'random' -d 'cifar10' -n 10 -top_k 2 -k 250

Starting Random Sampling Based Active Learning : 
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


The code is running on cuda:0 
Intialize training the model
Training ...


4it [00:00, 14.74it/s]

Train Epoch: 0, Train Size : 500	 Loss: 2.302171


...

Train Epoch: 39, Train Size : 1500	 Loss: 0.102104


24it [00:01, 23.49it/s]


====> Epoch: 39 Average loss: 0.0021
Evaluation ...


157it [00:04, 33.65it/s]

====> Accuracy: 40.64 
Updating the Training Data : 
All accuracies : 
[20.69, 36.97, 40.64]
